### Conversor de shp para json para utilizar a aoi local

In [1]:
#add

### Importação das bibliotecas e autenticação da GEE API

In [2]:
import ee
import geemap

try:
    ee.Initialize(project = 'teste-476811')
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project = 'teste-476811')

### Criação do mapa interativo e área de interesse

In [ ]:
Map = geemap.Map()
aoi = ee.FeatureCollection('projects/teste-476811/assets/Para')

### Criação do dataset

In [35]:
# Função para escolher coleção correta por ano
def landsat_collection_by_year(year):
    if (year < 1999) or (year == 2003):
        return 'LANDSAT/LT05/C02/T1_L2'   # Landsat 5
    else:
        return 'LANDSAT/LE07/C02/T1_L2'   # Landsat 7

dataset_cloudy = {}

# Cria coleções de 1988 a 2004 automaticamente
for year in range(1988, 2004):
    collection_id = landsat_collection_by_year(year)
    dataset_cloudy[str(year)] = (
        ee.ImageCollection(collection_id)
        .filterDate(f'{year}-01-01', f'{year}-12-31')
    )


### Máscara de nuvens e fator de escala

In [31]:
def cloudmask(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(1 << 3).eq(0)  # Nuvens
    saturationMask = image.select('QA_RADSAT').eq(0)  # Saturação
    return image.updateMask(qaMask).updateMask(saturationMask)

In [65]:
def scalefactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B6').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

### Visualização das imagens no mapa

In [66]:

ano = 2003

raw = dataset_cloudy[str(ano)].median().clip(aoi)
raw_visualization_params = {
    'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
    'min': 0,
    'max': 30000,
    'gamma': 1.4,
}

cloudfree = dataset_cloudy[str(ano)].map(cloudmask).map(scalefactors).median().clip(aoi)
cloudfree_visualization_params = {
    'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
    'min': 0,
    'max': 0.3,
    'gamma': 1.4,
}

lat, long = aoi.geometry().centroid().coordinates().get(1).getInfo(), aoi.geometry().centroid().coordinates().get(0).getInfo()
print(f'Centroide da AOI - Latitude: {lat}, Longitude: {long}')

Map.addLayer(raw, raw_visualization_params, f'{ano} - Cloudy')
Map.addLayer(cloudfree, cloudfree_visualization_params, f'{ano} - Cloud-free')
Map.addLayer(aoi.style(color = 'red', fillColor = '00000000'), {}, 'AOI')
Map.setCenter(long, lat, 6)
Map.set_zoom(6)
Map

Centroide da AOI - Latitude: -3.974512955502294, Longitude: -53.07056416496682


Map(bottom=8373.0, center=[-3.974512955502294, -53.07056416496682], controls=(WidgetControl(options=['position…